# Проект Автодополнение текстов

Вы работаете в соцсетевом приложении, где пользователи постят короткие тексты. В продукте стоит задача — добавить возможность автодополнения текстов. Разработчики просят вас создать модель, которую можно запускать на мобильных устройствах. Для смартфонов есть значительные требования по оперативной памяти и скорости работы, так что важна легковесность модели.

## Постановка задачи
**Создать нейросеть, которая на основе начала фразы предсказывает её продолжение**.

Поэтапное описание задачи:
1. Взять датасет от разработчиков, очистить его, подготовить для обучения модели.
1. Реализовать и обучить модель на основе рекуррентных нейронных сетей.
1. Замерить качество разработанной и обученной модели.
1. Взять более «тяжёлую» предобученную модель из Transformers и замерить её качество.
1. Проанализировать результаты и **дать рекомендации** разработчикам: стоит ли **использовать лёгкую модель или** лучше постараться поработать с ограничениями по памяти и **использовать большую предобученную**.

## Критерии успеха
- Используемые метрики качества: ROUGE-1 и ROUGE-2
- Минимальных порогов качества модели в задаче не задано


## Описание данных

датасет с короткими постами sentiment140

https://code.s3.yandex.net/deep-learning/tweets.txt

# Этап 0. Подготовка окружения

В задании требовалось весь код убрать в директории. Код доступен тут:

https://github.com/evgeny-ydsc/DLE-NLP

In [1]:
#!pip install -r requirements_sprint_2_project.txt

In [2]:
!pip install evaluate
!pip install rouge_score

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import evaluate
from drive.MyDrive.project1.src.data_utils import download_dataset
from drive.MyDrive.project1.src.next_token_dataset import prepare_loaders
from drive.MyDrive.project1.src.lstm_model import LSTMGenerator
from drive.MyDrive.project1.src.lstm_train_eval import ModelTrainer
from drive.MyDrive.project1.src.eval_transformer_pipeline import Distilgpt2Generator

# Этап 1. Сбор и подготовка данных

👷🚧🚧🚧🚧🚧 перед загрузкой на GPU заменить 200 на 20000

In [5]:
raw_data_path = "drive/MyDrive/project1/data/tweets.txt"
download_dataset("https://code.s3.yandex.net/deep-learning/tweets.txt",
                                 raw_data_path)
train_loader, val_loader, test_loader, tokenizer, val_texts, test_texts = \
    prepare_loaders (raw_data_path, batch_size=256, max_texts_count=10_000)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Этап 2. Реализация рекуррентной сети


In [6]:
model = LSTMGenerator(tokenizer.vocab_size)

# Этап 3. Тренировка модели

In [7]:
trainer = ModelTrainer (model, tokenizer)
trainer.train(train_loader, val_loader, val_texts, n_epochs=10)

100%|██████████| 371/371 [01:29<00:00,  4.15it/s]


Epoch 1 | Train Loss: 7.174 | Val Loss: 6.660 | Val Accuracy: 9.21%| rouge1: 0.002 | rouge2: 0.000


100%|██████████| 371/371 [01:27<00:00,  4.24it/s]


Epoch 2 | Train Loss: 6.210 | Val Loss: 6.375 | Val Accuracy: 10.55%| rouge1: 0.002 | rouge2: 0.000


100%|██████████| 371/371 [01:29<00:00,  4.14it/s]


...

Epoch 8 | Train Loss: 3.814 | Val Loss: 6.625 | Val Accuracy: 13.36%| rouge1: 0.007 | rouge2: 0.000


100%|██████████| 371/371 [01:29<00:00,  4.13it/s]


Epoch 9 | Train Loss: 3.517 | Val Loss: 6.757 | Val Accuracy: 13.10%| rouge1: 0.007 | rouge2: 0.000


100%|██████████| 371/371 [01:29<00:00,  4.14it/s]


Epoch 10 | Train Loss: 3.248 | Val Loss: 6.899 | Val Accuracy: 12.86%| rouge1: 0.008 | rouge2: 0.000


Видно, что начиная с 5й эпохи модель начала переобучатсья. Но GPU Практикума недоступна, а ресурс colab тоже уже закончился и не гарантирован (уже обрывался по полпути ранее). Поэтому не рискую его перезапускать ещё раз, оставлю модель как есть в этот раз.

In [8]:
rouge = evaluate.load("rouge")
for line in test_texts[:3]:
    l = len(line)
    i = int(l*0.75)
    start = line [:i]
    finish = line [i:]
    predicted_finish = model.generate_output_text(tokenizer, start, l-i)
    results = rouge.compute(predictions=[predicted_finish], references=[finish])
    print (f"----------------------------------------------")
    print (f"оригинальная строка: {line}")
    print (f"оригинальное начало: {start}")
    print (f"оригинальный конец : {finish}")
    print (f"предсказанный конец: {predicted_finish}")
    print (f"rouge1={results['rouge1']:.3f}, rouge2={results['rouge2']:.3f}\n")


----------------------------------------------
оригинальная строка: locked out of my outlook account for the 3rd time in 3 days
оригинальное начало: locked out of my outlook account for the 3rd
оригинальный конец :  time in 3 days
предсказанный конец: ineverandiandiam##icansoundsandfor##k##gfor
rouge1=0.000, rouge2=0.000

----------------------------------------------
оригинальная строка: back at the office still only 3 days until another long weekend
оригинальное начало: back at the office still only 3 days until anot
оригинальный конец : her long weekend
предсказанный конец: ofofand##es##pformeiandiandstillandineedme
rouge1=0.000, rouge2=0.000

----------------------------------------------
оригинальная строка: beachbassbone roomie was home all day all he had to do was scratch at the door comforter has to goto large laundromat machines
оригинальное начало: beachbassbone roomie was home all day all he had to do was scratch at the door comforter has t
оригинальный конец : o goto large 

# Этап 4. Использование предобученного трансформера

In [9]:
bert_model = Distilgpt2Generator()

Device set to use cuda:0


In [10]:
for line in test_texts[:3]:
    l = len(line)
    i = int(l*0.75)
    start = line [:i]
    finish = line [i:]
    predicted_finish = bert_model.generate_output_text(start, l-i)[l:]
    results = rouge.compute(predictions=[predicted_finish], references=[finish])
    print (f"----------------------------------------------")
    print (f"оригинальная строка: {line}")
    print (f"оригинальное начало: {start}")
    print (f"оригинальный конец : {finish}")
    print (f"предсказанный конец: {predicted_finish}")
    print (f"rouge1={results['rouge1']:.3f}, rouge2={results['rouge2']:.3f}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=15) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=16) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/te

----------------------------------------------
оригинальная строка: locked out of my outlook account for the 3rd time in 3 days
оригинальное начало: locked out of my outlook account for the 3rd
оригинальный конец :  time in 3 days
предсказанный конец:  I can handle that, but I’d like to go back and think about my experience in the game.


I like what I see in The Legend of Zelda: Breath of the Wild. Some of the things that I liked about this game are all that are going to make me appreciate it more than any other game that I've played. It’s an old-school Zelda game that I loved, and it’s a lot better than that.
The Legend of Zelda: Breath of the Wild is the most beautiful, thrilling Zelda game I played, and I like what it is. I’ll continue to enjoy playing, and I’ll get to experience it.
As for the story, I really enjoy the story. I can’t wait to see what you guys think.
The Legend of Zelda: Breath of the Wild is my favorite Zelda game I’ve seen.
You can check out the rest of the video

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=32) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


----------------------------------------------
оригинальная строка: back at the office still only 3 days until another long weekend
оригинальное начало: back at the office still only 3 days until anot
оригинальный конец : her long weekend
предсказанный конец: d a new one on the table. In the same office, I had a new one on the table. In the same office, I had a new one on the table. In the same office, I had a new one on the table. In the same office, I had a new one on the table. In the same office, I had a new one on the table. In the same office, and in the same office, I had a new one on the table. In the same office, and in the same office, I was the only one left. I was the only one left.
The first time I was at the office, I had a new one on the table. In the same office, and in the same office, I was the only one left. I was the only one left. I was the only one left. I was the only one left. I was the only one left. I was the only one left.
I was the only one left.
I was the o

# Этап 5. Формулирование выводов


- В соответствии с поставленной задачей(см. первый раздел)
    - была проведена подготовка данных и их разведочный анализ, в результате которых:
        - были исправлены имена полей, исправлены типы данных, удалены малополезные признаки (в т. ч. с учётом мультиколлинеарности), заполнены пропуски, удалены явные и неявные дубликаты, ...
    - была натренирована модель, предсказывающая ...
        - выбрана наиболее оптимальная модель - **...**, показавшая наилучшие результаты:
            - качество предсказания: ...
            - это удовлетворяет заданный критерий качества: ...